# Medical-cost Prediction Model
This analysis has been divided into three parts:
***
1. Data preparation and preprocessing
***
  - Analysing different sources of data
  - Aggregating data from different tables
  - Cleaning Data by rectifying data entry issues
  - Imputing missing values
  - Changing data types
  - Engineering a set of new features from the raw data<br>
***
2. Exploratory Data analysis
***
  - Exploring relationship between Categorical valiables and numeric target variable
  - Exploring relationship between numeric valiables and numeric target variable
  - Validating the relationship using statistical tests and visualizations relationship between Categorical valiables and numeric target variable<br>
***
3. Machine Learning Model fitting
***
  - Train test split
  - Model Defintion of candidate models
  - Model Fitting
  - Model Evaluation 
  - Model Tuning
  - Model Reevaluation
  - Feature Importance
      - Using Genetic Algorithm
      - Using Tree based feature importance based on Gain criteria
  - Model intuition using Decision tree plotting<br>
 

***
## Part 1: Data Preparation and preprocessing
***

In [4]:
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### 1. Reading data

In [4]:
bill_amount= pd.read_csv("D:/Python WD/codebase/data/Urmanpreet/bill_amount.csv")
bill_id= pd.read_csv("D:/Python WD/codebase/data/Urmanpreet/bill_id.csv")
clinical_data= pd.read_csv("D:/Python WD/codebase/data/Urmanpreet/clinical_data.csv")
demographics= pd.read_csv("D:/Python WD/codebase/data/Urmanpreet/demographics.csv")

### 2a) Inspecting the clinical data dataset

In [6]:
clinical_data.head(3)
clinical_data.shape
#3400,26
print("dataset has 3400 rows of data")
clinical_data.drop_duplicates('id').shape
print("3000 rows of unique customer data")
clinical_data.drop_duplicates(['id','date_of_admission']).shape
#3400
mask= (clinical_data.groupby('id').size()>1)
mask[mask.values==True][0:1]
print('inspecting sample customer')
clinical_data[clinical_data.id=='012c5eb5397a72f8aeb64f942b60846d']

,id,date_of_admission,date_of_discharge,medical_history_1,medical_history_2,medical_history_3,medical_history_4,medical_history_5,medical_history_6,medical_history_7,...,symptom_1,symptom_2,symptom_3,symptom_4,symptom_5,lab_result_1,lab_result_2,lab_result_3,weight,height
0,1d21f2be18683991eb93d182d6b2d220,2011-01-01,2011-01-11,0,1.0,0,0,0.0,0,0,...,0,0,0,1,1,13.2,30.9,123.0,71.3,161.0
1,62bdca0b95d97e99e1c712048fb9fd09,2011-01-01,2011-01-11,0,0.0,0,0,0.0,0,0,...,0,0,1,1,1,13.8,22.6,89.0,78.4,160.0
2,c85cf97bc6307ded0dd4fef8bad2fa09,2011-01-02,2011-01-13,0,0.0,0,0,0.0,0,0,...,1,1,1,1,0,11.2,26.2,100.0,72.0,151.0


(3400, 26)

dataset has 3400 rows of data


(3000, 26)

3000 rows of unique customer data


(3400, 26)

id
012c5eb5397a72f8aeb64f942b60846d    True
dtype: bool

inspecting sample customer


,id,date_of_admission,date_of_discharge,medical_history_1,medical_history_2,medical_history_3,medical_history_4,medical_history_5,medical_history_6,medical_history_7,...,symptom_1,symptom_2,symptom_3,symptom_4,symptom_5,lab_result_1,lab_result_2,lab_result_3,weight,height
301,012c5eb5397a72f8aeb64f942b60846d,2011-06-08,2011-06-23,0,0.0,0,0,0.0,1,0,...,1,1,1,0,0,15.2,25.4,84.0,99.2,171.0
3327,012c5eb5397a72f8aeb64f942b60846d,2015-11-17,2015-11-27,0,NaN,No,0,NaN,1,0,...,0,1,0,1,1,12.3,26.8,106.0,96.2,171.0


#### Inference:
**Dataset corresponds to 3000 unique customers meaning one customer may have admitted more than one time**

### 2b. Inspecting the bill_id dataset

In [321]:
bill_id.head(3)
bill_id.shape
#13600,3
#dataset has 3400 rows of data
bill_id.drop_duplicates('bill_id').shape
#13600 rows
bill_id.drop_duplicates('patient_id').shape
#3000 rows
#dataset corresponds to 3000 unique customers meaning one customer has on an average 4.5 bills
##exploring data of sample customer
bill_id[bill_id.patient_id=='012c5eb5397a72f8aeb64f942b60846d']
#the customer has two admissions and 8 bills in total


,bill_id,patient_id,date_of_admission
0,7968360812,1d21f2be18683991eb93d182d6b2d220,2011-01-01
1,6180579974,62bdca0b95d97e99e1c712048fb9fd09,2011-01-01
2,7512568183,1d21f2be18683991eb93d182d6b2d220,2011-01-01


(13600, 3)

(13600, 3)

(3000, 3)

,bill_id,patient_id,date_of_admission
1164,5062243503,012c5eb5397a72f8aeb64f942b60846d,2011-06-08
1165,9951536829,012c5eb5397a72f8aeb64f942b60846d,2011-06-08
1168,8573149856,012c5eb5397a72f8aeb64f942b60846d,2011-06-08
1171,5354388962,012c5eb5397a72f8aeb64f942b60846d,2011-06-08
13312,1595230701,012c5eb5397a72f8aeb64f942b60846d,2015-11-17
13313,6772496454,012c5eb5397a72f8aeb64f942b60846d,2015-11-17
13314,3385626862,012c5eb5397a72f8aeb64f942b60846d,2015-11-17
13315,1475841641,012c5eb5397a72f8aeb64f942b60846d,2015-11-17


#### Inference: 
- A patient has admitted multiple time 
- During each admission, a patient has multiple bills

### 2c. Inspecting the bill_amount dataset

In [322]:
bill_amount.head(3)
bill_amount.shape
#13600,3
#dataset has 3400 rows of data
bill_amount.drop_duplicates('bill_id').shape
#13600 rows

##### Inference:
dataset corresponds to 13600 unique bills

### 2d. Inspecting the demographics dataset

In [323]:
demographics.head(3)
demographics.shape
#3000

#### Inference:
It contains demograohics data for 3000 customers

## 3. Refined Problem Statement
- **Since, a patient has taken admission multiple times, and each time he might come for different disease. He also has different set of features like weight, medical history each time he visits**

- **Therefore, for better model fitting we are predicting the toal cost of treatment for each patient per admission for modelling and analysis purpose. Hence, Patientid- date_of_admission is the composite key**

- **Later if a patient has taken multiple admissions, his total cost can be aggregated**

### 4. Data preparation for our model
For our data, patient_id+ admission_date is the composite key.
Following data preparation steps are performed
#### Steps performed:
1. Take clinical data table as the base table and merge it with demograhics data
2. Take bill_id table and merge it with bill amount table
3. Find total bill amount for each paitent_id+date of admission
4. merge the resultant table to the clinical data table
##### A final table is obtained at the patient_id-date_of_admission level

In [8]:
dataset= clinical_data
dataset= pd.merge(dataset, demographics, left_on='id', right_on= 'patient_id', how='left')
temp= pd.merge(bill_id, bill_amount, left_on='bill_id', right_on='bill_id', how='left')
#aggregate bill amount at the patient id, date if admissino level
temp= temp.groupby(['patient_id','date_of_admission']).agg({'amount':sum}).reset_index()
##merge temp with base dataset for patient, date of admission combination
dataset= pd.merge(dataset, temp,left_on=['id','date_of_admission'] , right_on=['patient_id','date_of_admission'], how= 'left')
dataset= dataset.drop(columns= ['patient_id_y','patient_id_x']).rename(columns={'id':'patient_id'})
dataset.head(3)


,patient_id,date_of_admission,date_of_discharge,medical_history_1,medical_history_2,medical_history_3,medical_history_4,medical_history_5,medical_history_6,medical_history_7,...,lab_result_1,lab_result_2,lab_result_3,weight,height,gender,race,resident_status,date_of_birth,amount
0,1d21f2be18683991eb93d182d6b2d220,2011-01-01,2011-01-11,0,1.0,0,0,0.0,0,0,...,13.2,30.9,123.0,71.3,161.0,Male,Indian,Singaporean,1976-12-19,15405.418665
1,62bdca0b95d97e99e1c712048fb9fd09,2011-01-01,2011-01-11,0,0.0,0,0,0.0,0,0,...,13.8,22.6,89.0,78.4,160.0,Female,Malay,Singaporean,1978-07-20,23303.000924
2,c85cf97bc6307ded0dd4fef8bad2fa09,2011-01-02,2011-01-13,0,0.0,0,0,0.0,0,0,...,11.2,26.2,100.0,72.0,151.0,Female,Indian,Singaporean,1967-08-14,16813.123088


In [9]:
dataset.to_csv("D:/Python WD/codebase/data/merged_dataset.csv")



### 5. Data Cleaning 
1. Finding Missing values
2. Finding data issues

In [10]:
dataset.describe()
dataset['medical_history_3'].value_counts()

#### Observations:
1. medical history_2, 5 have missing values"
2. medical history columns have 'yes/no' in addition to 0,1

#### Treatment rule:
- Replace: Yes with 1, No with 0
- Replace missing value with 0 as missing data in medical history column is most likely to be absense of that disease

#### Assumption taken:
Missing data in symptom column is most likely to be no medical history

In [15]:

import numpy as np
dataset[['medical_history_1','medical_history_2','medical_history_3','medical_history_4','medical_history_5','medical_history_6','medical_history_7']]= dataset[['medical_history_1','medical_history_2','medical_history_3','medical_history_4','medical_history_5','medical_history_6','medical_history_7']].replace({'No':0,'Yes':1, np.nan: 0})


In [16]:
dataset.medical_history_2.unique()
dataset.head(20)

array([1., 0.])

,patient_id,date_of_admission,date_of_discharge,medical_history_1,medical_history_2,medical_history_3,medical_history_4,medical_history_5,medical_history_6,medical_history_7,...,lab_result_1,lab_result_2,lab_result_3,weight,height,gender,race,resident_status,date_of_birth,amount
0,1d21f2be18683991eb93d182d6b2d220,2011-01-01,2011-01-11,0,1.0,0,0,0.0,0,0,...,13.2,30.9,123.0,71.3,161.0,Male,Indian,Singaporean,1976-12-19,15405.418665
1,62bdca0b95d97e99e1c712048fb9fd09,2011-01-01,2011-01-11,0,0.0,0,0,0.0,0,0,...,13.8,22.6,89.0,78.4,160.0,Female,Malay,Singaporean,1978-07-20,23303.000924
2,c85cf97bc6307ded0dd4fef8bad2fa09,2011-01-02,2011-01-13,0,0.0,0,0,0.0,0,0,...,11.2,26.2,100.0,72.0,151.0,Female,Indian,Singaporean,1967-08-14,16813.123088
3,e0397dd72caf4552c5babebd3d61736c,2011-01-02,2011-01-14,0,1.0,0,0,0.0,1,1,...,13.3,28.4,76.0,64.4,152.0,Female,Chinese,PR,1967-12-06,27994.878530
4,94ade3cd5f66f4584902554dff170a29,2011-01-08,2011-01-16,0,0.0,0,0,0.0,1,1,...,12.0,27.8,87.0,55.6,160.0,Female,Others,PR,1982-07-18,10707.950907
5,59e07adc2dbc5f70131f57d003610d74,2011-01-07,2011-01-17,0,0.0,0,0,0.0,0,0,...,15.8,31.0,75.0,78.8,169.0,Male,Chinese,Singaporean,1974-01-14,18045.365209
6,f5c4d97ebf32d49967fbf4f6c5fd52ec,2011-01-06,2011-01-17,0,0.0,0,0,0.0,0,1,...,12.1,23.0,83.0,81.8,164.0,Female,Malay,Singapore citizen,1934-04-24,21590.122425
7,1e788744568c21b390c5aa8c5dd61335,2011-01-07,2011-01-17,0,0.0,0,0,0.0,0,1,...,16.4,26.8,126.0,73.5,173.0,Male,Chinese,Singaporean,1961-12-23,9612.227445
8,457402b26562d41f4e40906d3d17d5d1,2011-01-12,2011-01-18,0,0.0,0,0,0.0,0,0,...,12.5,32.9,87.0,98.4,166.0,m,Chinese,Singaporean,1967-06-04,13420.444483
9,79f52395dab0e6d3a03c48f765cb6562,2011-01-02,2011-01-18,0,0.0,0,0,0.0,0,1,...,12.1,23.6,109.0,92.8,176.0,Male,Chinese,Singapore citizen,1936-04-05,17153.905421


In [17]:
#Gender column
dataset.gender.unique()
dataset.resident_status.unique()
dataset.race.unique()

array(['Male', 'Female', 'm', 'f'], dtype=object)

array(['Singaporean', 'PR', 'Singapore citizen', 'Foreigner'],
      dtype=object)

array(['Indian', 'Malay', 'Chinese', 'Others', 'chinese', 'India'],
      dtype=object)

#### Data issues: Observations  
1. **Gender** has m and f extra,
    - **Treatment**: We replace with male and feamale
2. **resident_status** has Singaporean and Singapore citizen,
    - **Treatment**: We replace Singapore citizen with Singaporean
3. **race** has both chinese and Chinese, India and Indian
    - **Treatment**: We replace chinese with Chinese, India with Indian

In [18]:
import numpy as np
dataset['gender']= dataset['gender'].replace({'m':'Male','f':'Female'})
dataset['resident_status']= dataset['resident_status'].replace({'Singapore citizen':'Singaporean'})
dataset['race']= dataset['race'].replace({'chinese':'Chinese','India': 'Indian'})

### 6. Data type conversion

In [19]:
dataset.dtypes
import datetime
#convert datatypes
dataset[['gender','race','resident_status']]=dataset[['gender','race','resident_status']].astype('category')
dataset[['gender','race','resident_status']]=dataset[['gender','race','resident_status']].astype('category')
dataset[['medical_history_2','medical_history_3','medical_history_5']]= dataset[['medical_history_2','medical_history_3','medical_history_5']].astype('int64')
dataset[['date_of_admission','date_of_discharge','date_of_birth']]= dataset[['date_of_admission','date_of_discharge','date_of_birth']].apply(pd.to_datetime)
dataset['amount']= round(dataset['amount'],2)
dataset.dtypes

patient_id             object
date_of_admission      object
date_of_discharge      object
medical_history_1       int64
medical_history_2     float64
medical_history_3      object
medical_history_4       int64
medical_history_5     float64
medical_history_6       int64
medical_history_7       int64
preop_medication_1      int64
preop_medication_2      int64
preop_medication_3      int64
preop_medication_4      int64
preop_medication_5      int64
preop_medication_6      int64
symptom_1               int64
symptom_2               int64
symptom_3               int64
symptom_4               int64
symptom_5               int64
lab_result_1          float64
lab_result_2          float64
lab_result_3          float64
weight                float64
height                float64
gender                 object
race                   object
resident_status        object
date_of_birth          object
amount                float64
dtype: object

patient_id                    object
date_of_admission     datetime64[ns]
date_of_discharge     datetime64[ns]
medical_history_1              int64
medical_history_2              int64
medical_history_3              int64
medical_history_4              int64
medical_history_5              int64
medical_history_6              int64
medical_history_7              int64
preop_medication_1             int64
preop_medication_2             int64
preop_medication_3             int64
preop_medication_4             int64
preop_medication_5             int64
preop_medication_6             int64
symptom_1                      int64
symptom_2                      int64
symptom_3                      int64
symptom_4                      int64
symptom_5                      int64
lab_result_1                 float64
lab_result_2                 float64
lab_result_3                 float64
weight                       float64
height                       float64
gender                      category
r

### 7. Feature Engineering
Six features have been defined as follows: 
1. **age_at_admission**: date of admission- date of birth 
2. **age_band**: upto 30, 30 to 50, 50
3. **bmi**: weight(kg)/[height(cm)/100]^2
4. **duration_of_stay**: date of discharge-date of admission
5. **count_of_medical_history**: count of type of medical hostories
6. **bmi_band**: upto 18.5, 18.5 to 24.9, 25 to 29.9, 30 and above


In [20]:
#1) age at admission
dataset['age_at_admission']= round((dataset['date_of_admission']- dataset['date_of_birth']).dt.days/365.25)

#2) #age band feature
dataset.loc[dataset['age_at_admission']<=30,'age_band']='young_aged'
dataset.loc[(dataset['age_at_admission']>30) & (dataset['age_at_admission']<=50),'age_band']='middle_aged'
dataset.loc[dataset['age_at_admission']>50,'age_band']='old_aged'
dataset[['age_band']]=dataset[['age_band']].astype('category')


#3) bmi
dataset['bmi']=round(dataset['weight']/np.power(dataset['height']/100,2),2)

#4) duration of stay
dataset['duration_of_stay']= round((dataset['date_of_discharge']- dataset['date_of_admission']).dt.days)

#5) count_of_medical_history
dataset['count_of_medical_history']= dataset.medical_history_1+dataset.medical_history_2+ dataset.medical_history_3+ dataset.medical_history_4+ dataset.medical_history_5+ dataset.medical_history_6+ dataset.medical_history_7

#6) bmi band
dataset.loc[dataset['bmi']<18.5,'bmi_band']='underweight'
dataset.loc[(dataset['bmi']>=18.5) & (dataset['bmi']<25),'bmi_band']='normal'
dataset.loc[(dataset['bmi']>=25) & (dataset['bmi']<30),'bmi_band']='overweight'
dataset.loc[dataset['bmi']>=30,'bmi_band']='obese'
dataset[['bmi_band']]=dataset[['bmi_band']].astype('category')


In [21]:
dataset

,patient_id,date_of_admission,date_of_discharge,medical_history_1,medical_history_2,medical_history_3,medical_history_4,medical_history_5,medical_history_6,medical_history_7,...,race,resident_status,date_of_birth,amount,age_at_admission,age_band,bmi,duration_of_stay,count_of_medical_history,bmi_band
0,1d21f2be18683991eb93d182d6b2d220,2011-01-01,2011-01-11,0,1,0,0,0,0,0,...,Indian,Singaporean,1976-12-19,15405.42,34.0,middle_aged,27.51,10,1,overweight
1,62bdca0b95d97e99e1c712048fb9fd09,2011-01-01,2011-01-11,0,0,0,0,0,0,0,...,Malay,Singaporean,1978-07-20,23303.00,32.0,middle_aged,30.62,10,0,obese
2,c85cf97bc6307ded0dd4fef8bad2fa09,2011-01-02,2011-01-13,0,0,0,0,0,0,0,...,Indian,Singaporean,1967-08-14,16813.12,43.0,middle_aged,31.58,11,0,obese
3,e0397dd72caf4552c5babebd3d61736c,2011-01-02,2011-01-14,0,1,0,0,0,1,1,...,Chinese,PR,1967-12-06,27994.88,43.0,middle_aged,27.87,12,3,overweight
4,94ade3cd5f66f4584902554dff170a29,2011-01-08,2011-01-16,0,0,0,0,0,1,1,...,Others,PR,1982-07-18,10707.95,28.0,young_aged,21.72,8,2,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3395,4368c266084e640c469a5d41b339805c,2015-12-26,2016-01-07,0,0,0,0,0,0,0,...,Chinese,Singaporean,1964-01-05,20730.38,52.0,old_aged,28.51,12,0,overweight
3396,4f67a54ab205cc9e7e2b0a4ee08e4fba,2015-12-28,2016-01-07,0,0,0,0,0,1,0,...,Chinese,Singaporean,1970-08-07,19356.77,45.0,middle_aged,30.26,10,1,obese
3397,ac52a32f8ce8c46d82df2d72052ae5a9,2015-12-28,2016-01-08,0,0,0,0,0,1,0,...,Chinese,Singaporean,1979-03-06,20141.41,37.0,middle_aged,26.27,11,1,overweight
3398,5189c62dfb8e6075b5c7fd4766671bd4,2015-12-26,2016-01-09,0,1,0,0,0,0,0,...,Chinese,Singaporean,1960-05-10,10932.66,56.0,old_aged,35.87,14,1,obese


### 8. Export the data prepared to a dump file

In [22]:
import joblib, os
joblib.dump(dataset,"D:\\Python WD\\codebase\\model_objects\\dataset_feature_engineered")
#ds2=joblib.load("D:\\Python WD\\codebase\\model_objects\\dataset_feature_engineered")


['D:\\Python WD\\codebase\\model_objects\\dataset_feature_engineered']